In [1]:
import os
from chatlas import ChatGoogle
from chatlas import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
load_dotenv()

True

In [2]:
import pandas as pd

In [3]:
# chat = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
chat = ChatOpenAI(
    model="llama-3",
    api_key=os.getenv("LLAMA_API_KEY"),
    base_url="https://traip13.tgptinf.ucsd.edu/v1",
)

🌍 Tool Classes for Environmental Intelligence
---------------------------------------------
Used to get:
- Latitude & longitude from location names
- Current temperature & wind speed from coordinates
- Recent news articles based on product + region

Registered with the LLM so it can trigger them automatically.


#### 🌡️ Tool implementation: Gets real-time weather and wind data from Open-Meteo based on coordinates


In [4]:
class GetCurrentTemperature(BaseModel):
    """
    Get the current weather given a latitude and longitude.
    You MUST provide both latitude and longitude as numbers.
    If the user provides a location name (not latitude/longitude),
    you MUST first call the 'get_lat_long' tool to obtain the coordinates
    then call this tool with those coordinates.
    """

    latitude: float = Field(
        description="The latitude of the location. Must be a float."
    )
    longitude: float = Field(
        description="The longitude of the location. Must be a float."
    )

In [5]:
def get_current_temperature(latitude: float, longitude: float) -> dict:
    lat_lng = f"latitude={latitude}&longitude={longitude}"
    url = f"https://api.open-meteo.com/v1/forecast?{lat_lng}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    response = requests.get(url)
    json = response.json()
    return json["current"]

#### 📍 Tool implementation: Converts a location name into latitude and longitude using maps.co


In [6]:
class GetLatLong(BaseModel):
    """
    Use this tool to get the latitude and longitude for a location name provided by the user.
    If the user asks for weather and only provides a location name, call this tool first to get coordinates.
    """

    location: str = Field(
        description="The location name to get latitude and longitude for. Must be a string."
    )


In [7]:
def get_lat_long(location: str) -> dict:
    url = f"https://geocode.maps.co/search?q='{location}'&api_key={os.getenv('GEO_API_KEY')}"
    response = requests.get(url)
    json = response.json()
    if len(json) == 0:
        raise ValueError(
            f"Could not find location: {location}. Try to determine the location from the LLM response and call this tool again with the new location."
        )
    else:
        return {"latitude": float(json[0]["lat"]), "longitude": float(json[0]["lon"])}

#### 🌐 Tool implementation: Fetches recent news from NewsAPI based on region and product keyword


In [8]:
class GetNews(BaseModel):
    """
    Use this tool to get recent news headlines related to a region and product.
    You MUST provide both a region (e.g., 'Asia') and a product (e.g., 'steel').
    This tool uses the NewsAPI.org service.
    """

    region: str = Field(
        description="The region to search news for. Must be a string. Example: 'Asia'"
    )
    product: str = Field(
        description="The product to search news for. Must be a string. Example: 'steel'"
    )

In [9]:
import requests
import os

news_key = os.getenv("news_key")

def get_news(region: str, product: str):
    """
    Get recent news headlines related to a region and product.
    """
    query = f"{product} {region}"
    url = f"https://newsapi.org/v2/everything?q={query}&sortBy=publishedAt&pageSize=5&apiKey={news_key}"

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code} - {response.text}")
    
    articles = response.json().get("articles", [])
    return [
        {
            "title": a["title"],
            "url": a["url"],
            "publishedAt": a["publishedAt"]
        }
        for a in articles
    ]

In [10]:
chat.register_tool(get_lat_long, model=GetLatLong)
chat.register_tool(get_current_temperature, model=GetCurrentTemperature)
chat.register_tool(get_news, model=GetNews)


In [11]:
_ = chat.chat(
    "what is weather in beijing ",
)

<br>



```python
# 🔧 tool request (chatcmpl-tool-e3fb1dd5fd1b4415b986301c138a407e)
GetLatLong(location=beijing)
```




```python
# ✅ tool result (chatcmpl-tool-e3fb1dd5fd1b4415b986301c138a407e)
{'latitude': 39.9057136, 'longitude': 116.3912972}
```

<br>



```python
# 🔧 tool request (chatcmpl-tool-bd9739b86811481dae01b97c3dd22392)
GetCurrentTemperature(latitude=39.9057136, longitude=116.3912972)
```




```python
# ✅ tool result (chatcmpl-tool-bd9739b86811481dae01b97c3dd22392)
{'time': '2025-06-03T22:30', 'interval': 900, 'temperature_2m': 18.9, 'wind_speed_10m': 3.1}
```

<br>

The current temperature in Beijing is 18.9 degrees Celsius.

In [12]:
_ = chat.chat(
    "We are planning to procure steel from asian market, tell me if theres is any geo politics mights affect our process, list the risk factors in bullet points"
)

<br>



```python
# 🔧 tool request (chatcmpl-tool-9e4bbed911054035a601da6a817f04a1)
GetNews(region=Asia, product=steel)
```




```python
# ✅ tool result (chatcmpl-tool-9e4bbed911054035a601da6a817f04a1)
[{'title': "The first task facing South Korea's next leader: Handling Trump", 'url': 'https://www.nbcnews.com/world/asia/south-korea-trump-presidential-election-lee-jae-myung-yoon-kim-japan-rcna210302', 'publishedAt': '2025-06-02T22:00:00Z'}, {'title': 'ERIC KIM VIRAL COOKBOOK 101', 'url': 'https://erickimphotography.com/blog/2025/06/02/eric-kim-viral-cookbook-101/', 'publishedAt': '2025-06-02T21:48:01Z'}, {'title': 'Asia’s Top-Rated Bonds Lifted by Pullback From US Treasuries', 'url': 'https://biztoc.com/x/b42a3e123d9e3500', 'publishedAt': '2025-06-02T20:26:01Z'}, {'title': 'Potential Upside Scenarios for Copper Prices in 2026', 'url': 'https://oilprice.com/Metals/Commodities/Potential-Upside-Scenarios-for-Copper-Prices-in-2026.html', 'publishedAt': '2025-06-02T19:00:00Z'}, {'title': 'Tariful pentru oțel majorat de Trump afectează mai mult industria americană decât Europa', 'url': 'https://www.mediafax.ro/economic/tariful-pentru-otel-majorat-de-trump-afecteaza-mai-mult-industria-americana-decat-europa-23561462', 'publishedAt': '2025-06-02T16:51:39Z'}]
```

<br>

Based on the news headlines, here are some potential risk factors that could affect your steel procurement process in the Asian market:

* Trade tensions and tariffs: The US-China trade war and the recent tariffs imposed by the US on steel imports could impact the availability and pricing of steel in the Asian market.
* Geopolitical instability: The ongoing tensions between North and South Korea, as well as the recent presidential election in South Korea, could potentially disrupt trade and commerce in the region.
* Market fluctuations: The pullback from US Treasuries and the potential upside scenarios for copper prices in 2026 could impact the steel market and prices in the Asian region.
* Regulatory changes: Changes in trade policies and regulations, such as the recent tariffs imposed by the US, could affect the steel industry and impact your procurement process.
* Industry trends: The steel industry is subject to various trends and factors, such as changes in demand, supply chain disruptions, and technological advancements, which could impact your procurement process.

It's important to note that these risk factors are based on current news headlines and may not be exhaustive or up-to-date. It's always a good idea to conduct thorough research and consult with experts in the field to get a more comprehensive understanding of the potential risks and challenges involved in steel procurement in the Asian market.

📋 Tool: Generate RFQ Pre-Evaluation Questions
----------------------------------------------
LLM-based prompt to generate thoughtful, context-aware questions a procurement team
should ask before sending an RFQ to a vendor. Includes logistics, risk, and compliance checks.

In [14]:
class GenerateRFQPrepQuestions(BaseModel):
    vendor: str = Field(description="The name of the vendor")
    product: str = Field(description="The product for which RFQ is being prepared")
    location: str = Field(description="The vendor's location")


In [15]:
def generate_rfq_prep_questions(vendor: str, product: str, location: str) -> list:
    prompt = (
        f"You are a procurement expert. A vendor named '{vendor}' based in '{location}' "
        f"is being considered for supplying '{product}'. Generate a list of questions that a sourcing team should ask "
        f"before sending an RFQ, considering logistics, risk, past performance, and alternatives. Limit the questions to 5"
        f"Keep the questions concise and focused on critical aspects that could impact the procurement process."
        
    )
    return chat.chat(prompt)  # Use your LLM call


Give a file and try!

In [16]:
generate_rfq_prep_questions(
    vendor="ABC Cable Co.",
    product= "PLUMBING VALVES",
    location="Hanoi, Vietnam"
)

<br>

Here are 5 questions that a sourcing team should ask before sending an RFQ to ABC Cable Co.:

1. What is your experience in manufacturing and supplying plumbing valves, and can you provide references from previous customers?
2. What are your logistics and shipping capabilities, and how will you ensure timely delivery of products from Hanoi, Vietnam to our location?
3. What quality control measures do you have in place to ensure that your plumbing valves meet our required standards and specifications?
4. What is your pricing strategy, and are you willing to provide a competitive pricing structure for large volume orders?
5. Do you have any certifications (e.g. ISO 9001) or compliance with international standards (e.g. NSF, UL) that demonstrate your commitment to quality and regulatory adherence?

In [17]:
scope_doc = pd.read_csv("scope_document-RFQ.csv")

In [18]:
scope_doc

Item Name          Specification  Quantity Unit of Measure  \
0         earthing strip     Copper, 25mm x 3mm       450             MTR   
1          control cable       XLPE 2C x 4 sqmm       888             MTR   
2         signal conduit         HDPE, 50mm dia       346             MTR   
3             cable tray        Ladder Type, GI       819             MTR   
4  structural steel beam  U-Channel, Galvanized       593              EA   

  Delivery Date  Drawing Ref                   Recommended Vendor   Region  \
0       8/30/25  DM-ELEC-001  Spain Earthing Strip Solutions Ltd.    Spain   
1       8/30/25  DM-ELEC-009         Spain Flanges Solutions Ltd.    Spain   
2       8/30/25   DM-SIG-004         Spain Flanges Solutions Ltd.    Spain   
3       8/30/25  DM-ELEC-005               Vietnam Solutions Ltd.  Vietnam   
4       8/30/25  DM-STRC-006               Vietnam Solutions Ltd.  Vietnam   

                                       Justification  \
0  Spain Earthing Strip Solutions Ltd. is selecte...   
1  Spain Flanges Solutions Ltd. is selected due t...   
2  Spain Flanges Solutions Ltd. is a suitable ven...   
3  Vietnam Control Cable Solutions Ltd. is recomm...   
4  Vietnam Flanges Solutions Ltd. is a suitable v...   

                            Validation Errors  
0  Missing required certification (ISO/CE/UL)  
1                                         NaN  
2                                         NaN  
3                                         NaN  
4                                         NaN

In [19]:
scope_doc.columns

Index(['Item Name', 'Specification', 'Quantity', 'Unit of Measure',
       'Delivery Date', 'Drawing Ref', 'Recommended Vendor', 'Region',
       'Justification', 'Validation Errors'],
      dtype='object')

AI-Based RFQ Email Generator
----------------------------

This script analyzes a scope document (CSV file), groups procurement items by vendor, 
and uses a language model to generate tailored RFQ emails.

Steps:
1. Load and parse the scope_doc DataFrame.
2. Group items by vendor using a helper function.
3. Generate AI-powered professional RFQ emails per vendor.
4. If validation errors (e.g., missing certifications) exist, the AI includes a polite request.


#### 🔧 Helper function to group items by vendor from the scope_doc DataFrame.
#### It returns a dictionary with vendor names as keys and a list of their items as values.


In [20]:
def prepare_vendor_item_summary(df):
    grouped = df.groupby("Recommended Vendor")
    vendor_data = {}

    for vendor, group_df in grouped:
        items = group_df.to_dict(orient="records")
        region = items[0].get("Region", "")
        vendor_data[vendor] = {
            "region": region,
            "items": items
        }

    return vendor_data


#### 🤖 Generates a complete RFQ email using AI for a specific vendor.
#### It summarizes all items, delivery info, and appends a certification request if validation errors exist.
#### The output is a ready-to-send email text.


In [21]:
def generate_ai_email_for_vendor(vendor_name, region, items):
    item_descriptions = ""
    validation_summary = []

    for item in items:
        item_descriptions += (
            f"- {item['Item Name']}: {item['Specification']}, {item['Quantity']} {item['Unit of Measure']}, "
            f"Delivery by {item['Delivery Date']}\n"
        )
        if item.get("Validation Errors") and isinstance(item["Validation Errors"], str):
            validation_summary.append(item["Validation Errors"].strip())

    prompt = f"""
You are a procurement assistant.

Generate a professional RFQ email to the vendor named '{vendor_name}' located in {region}. 
The email should request a quotation for the following items:

{item_descriptions}

If any validation errors are mentioned, include a line requesting clarification or documentation.
Validation issues: {', '.join(set(validation_summary)) if validation_summary else 'None'}

Keep the tone polite and formal. Do not include prices—just a request for quotation, availability, and certifications.
Close the email professionally.
"""

    return chat.chat(prompt).content.strip()


#### 🔁 Main wrapper to iterate over all vendors and generate AI-driven emails.
#### This combines the grouping logic and AI prompt generation in one pass.


In [22]:
def generate_all_rfq_emails_ai(scope_doc):
    vendor_summary = prepare_vendor_item_summary(scope_doc)
    emails = {}

    for vendor, data in vendor_summary.items():
        email = generate_ai_email_for_vendor(vendor, data["region"], data["items"])
        emails[vendor] = email

    return emails


In [23]:
emails = generate_all_rfq_emails_ai(scope_doc)

for vendor, email in emails.items():
    print(f"\n📨 Email to: {vendor}\n{'='*80}\n{email}\n")

<br>

Subject: Request for Quotation for Earthing Strip

Dear Spain Earthing Strip Solutions Ltd.,

I hope this email finds you well. I am reaching out to request a quotation for the following item:

* Earthing strip: Copper, 25mm x 3mm, 450 MTR, with a required delivery date of August 30, 2025.

Could you please provide us with your best quotation for the above-mentioned item, including availability and lead time? We are also interested in knowing if you have any certifications (e.g. ISO, CE, UL) that demonstrate your commitment to quality and regulatory adherence.

However, our initial review of your company's profile has indicated a validation issue - we were unable to verify the required certification (ISO/CE/UL). Could you please clarify or provide documentation to support your certification status?

We look forward to receiving your quotation and discussing how we can work together. Please feel free to contact me if you have any questions or concerns.

Thank you for your time and consideration.

Best regards,
[Your Name]
Procurement Assistant

<br>

Subject: Request for Quotation for Control Cable and Signal Conduit

Dear Spain Flanges Solutions Ltd.,

I hope this email finds you well. I am reaching out to request a quotation for the following items:

* Control cable: XLPE 2C x 4 sqmm, 888 MTR, with a required delivery date of August 30, 2025.
* Signal conduit: HDPE, 50mm dia, 346 MTR, with a required delivery date of August 30, 2025.

Could you please provide us with your best quotation for the above-mentioned items, including availability and lead time? We are also interested in knowing if you have any certifications (e.g. ISO, CE, UL) that demonstrate your commitment to quality and regulatory adherence.

We have reviewed your company's profile and did not find any validation issues. However, please feel free to provide any additional information or documentation that you think would be relevant to our procurement process.

We look forward to receiving your quotation and discussing how we can work together. Please feel free to contact me if you have any questions or concerns.

Thank you for your time and consideration.

Best regards,
[Your Name]
Procurement Assistant

<br>

Subject: Request for Quotation for Cable Tray and Structural Steel Beam

Dear Vietnam Solutions Ltd.,

I hope this email finds you well. I am reaching out to request a quotation for the following items:

* Cable tray: Ladder Type, GI, 819 MTR, with a required delivery date of August 30, 2025.
* Structural steel beam: U-Channel, Galvanized, 593 EA, with a required delivery date of August 30, 2025.

Could you please provide us with your best quotation for the above-mentioned items, including availability and lead time? We are also interested in knowing if you have any certifications (e.g. ISO, CE, UL) that demonstrate your commitment to quality and regulatory adherence.

We have reviewed your company's profile and did not find any validation issues. However, please feel free to provide any additional information or documentation that you think would be relevant to our procurement process.

We look forward to receiving your quotation and discussing how we can work together. Please feel free to contact me if you have any questions or concerns.

Thank you for your time and consideration.

Best regards,
[Your Name]
Procurement Assistant


📨 Email to: Spain Earthing Strip Solutions Ltd.
Subject: Request for Quotation for Earthing Strip

Dear Spain Earthing Strip Solutions Ltd.,

I hope this email finds you well. I am reaching out to request a quotation for the following item:

* Earthing strip: Copper, 25mm x 3mm, 450 MTR, with a required delivery date of August 30, 2025.

Could you please provide us with your best quotation for the above-mentioned item, including availability and lead time? We are also interested in knowing if you have any certifications (e.g. ISO, CE, UL) that demonstrate your commitment to quality and regulatory adherence.

However, our initial review of your company's profile has indicated a validation issue - we were unable to verify the required certification (ISO/CE/UL). Could you please clarify or provide documentation to support your certification status?

We look forward to receiving your quotation and discussing how we can work together. Please feel free to contact me if you have any question